# Building a Crew to Prepare for Meetings

### Install Dependencies

In [ ]:
# Create reusable loading animation class
import os
import sys
import time
import threading

class LoadingAnimation:
    def __init__(self):
        self.stop_event = threading.Event()
        self.animation_thread = None

    def _animate(self, message="Loading"):
        chars = "/—\\|"
        while not self.stop_event.is_set():
            for char in chars:
                sys.stdout.write('\r' + message + '... ' + char)
                sys.stdout.flush()
                time.sleep(0.1)
                if self.stop_event.is_set():
                    sys.stdout.write("\n")
                    break

    def start(self, message="Loading"):
        self.stop_event.clear()
        self.animation_thread = threading.Thread(target=self._animate, args=(message,))
        self.animation_thread.daemon = True
        self.animation_thread.start()

    def stop(self, completion_message="Complete"):
        self.stop_event.set()
        if self.animation_thread:
            self.animation_thread.join()
        print(f"\r{completion_message} ✓")

# Use the animation for pip install
loader = LoadingAnimation()
loader.start("Installing")
%pip install -r requirements.txt -q
loader.stop("Installation complete")

### Helper Functions

In [2]:
import dotenv
from dotenv import dotenv_values

# Define a fake `load_dotenv` function
def _load_dotenv(*args, **kwargs):
    env_path = kwargs.get('dotenv_path', '.env')  # Default to '.env'
    parsed_env = dotenv_values(env_path)

    # Manually set valid key-value pairs
    for key, value in parsed_env.items():
        if key and value:  # Check for valid key-value pairs
            os.environ[key] = value

dotenv.load_dotenv = _load_dotenv

### Initialization and Setup
Initial imports for the CrewAI Flow and Crew and setting up the environment

In [ ]:
# Importing necessary libraries
import os
import yaml

# Importing Crew related components
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool, ScrapeWebsiteTool

# Apply a patch to allow nested asyncio loops in Jupyter
import nest_asyncio
nest_asyncio.apply()

In [4]:
# Create output directory if it doesn't exist
os.makedirs('output', exist_ok=True)

# Load agent and task configurations from YAML files
with open('config/agents.yml', 'r') as f:
    agents_config = yaml.safe_load(f)

with open('config/tasks.yml', 'r') as f:
    tasks_config = yaml.safe_load(f)

# Define the agents for our meeting preparation crew
researcher = Agent(
    role=agents_config['researcher']['role'],
    goal=agents_config['researcher']['goal'],
    backstory=agents_config['researcher']['backstory'],
    verbose=True,
    tools=[SerperDevTool(), ScrapeWebsiteTool()]  # Web search and document analysis tools
)

analyst = Agent(
    role=agents_config['analyst']['role'],
    goal=agents_config['analyst']['goal'],
    backstory=agents_config['analyst']['backstory'],
    verbose=True,
)

# Define the tasks for our crew
research_task = Task(
    description=tasks_config['research_task']['description'],
    expected_output=tasks_config['research_task']['expected_output'],
    agent=researcher,
)

preparation_task = Task(
    description=tasks_config['preparation_task']['description'],
    expected_output=tasks_config['preparation_task']['expected_output'],
    agent=analyst,
    output_file="output/meeting_brief.md"
)

# Create the crew
meeting_prep_crew = Crew(
    agents=[researcher, analyst],
    tasks=[research_task, preparation_task],
    process=Process.sequential,
    verbose=True
)

In [ ]:
# If inputs are not provided, ask for them
meeting_topic = input("Enter the meeting topic: ")
participants = input("Enter the meeting participants: ")

print(f"\nPreparing for meeting about: {meeting_topic}")
print(f"Participants: {participants}\n")

# Run the crew with the provided inputs
result = meeting_prep_crew.kickoff(inputs={
    "topic": meeting_topic,
    "participants": participants
})

print("\n\nMeeting brief has been saved to output/meeting_brief.md")
# Display the markdown content in a formatted way
from IPython.display import Markdown, display

# Display the raw result as formatted markdown
display(Markdown(result.raw))